In [0]:
# Lenguaje del Notebook: Python
# VERSIÓN 14: Unificador de Evidencias (Idempotente y Transaccional)

# MAGIC %md
# MAGIC # 📥 5. Unificar Evidencias (Staging -> Final) (v14)
# MAGIC 
# MAGIC Este notebook se ejecuta DESPUÉS de todas las tareas paralelas de validación.
# MAGIC Es idempotente y seguro ante fallos.
# MAGIC 
# MAGIC **Lógica (Transaccional):**
# MAGIC 1.  `main()` orquesta el proceso.
# MAGIC 2.  Lee `table_config` para encontrar todas las tablas `staging_evidences_table`.
# MAGIC 3.  Las une dinámicamente (`unionByName`) para crear un DataFrame `df_staging_completo`.
# MAGIC 4.  **Anti-Duplicación:** Obtiene los `execution_id`s de staging y los cruza con
# MAGIC     `dq_evidences` para encontrar los IDs que *aún no* han sido procesados.
# MAGIC 5.  Filtra `df_staging_completo` para quedarse solo con los datos nuevos.
# MAGIC 6.  **`APPEND`** los datos nuevos a la tabla final `dq_evidences`.
# MAGIC 7.  **Limpieza:** Si el `APPEND` es exitoso, `DELETE` los `execution_id`s procesados
# MAGIC     de todas las tablas de staging.

# COMMAND ----------

# DBTITLE 1, 1. Imports y Constantes
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, broadcast
from functools import reduce
from delta.tables import DeltaTable

# --- Widgets ---
dbutils.widgets.text("catalog_name", "workspace", "Catálogo de UC donde residen las tablas")
dbutils.widgets.text("schema_name", "framework_dq", "Esquema de UC donde residen las tablas")
# COMMAND ----------

# DBTITLE 2, 2. Carga de librerías y definición de constantes/mapas

CATALOG = dbutils.widgets.get("catalog_name")
SCHEMA = dbutils.widgets.get("schema_name")

TABLE_CONFIG = f"{CATALOG}.{SCHEMA}.dq_tables_config"
EVIDENCES_TABLE = f"{CATALOG}.{SCHEMA}.dq_evidences"


# COMMAND ----------

# DBTITLE 2, 2. Función Principal (main)
def main():
    """
    Flujo principal de unificación de evidencias (v14).
    """
    print("--- Iniciando Script de Unificación de Evidencias (v14) ---")
    
    try:
        # --- 1. Leer la Configuración Maestra ---
        print(f"Leyendo la configuración de maestros desde: {TABLE_CONFIG}")
        
        master_configs = (spark.table(TABLE_CONFIG)
                          .select("table_name", "staging_evidences_table")
                          .collect())
        
        if not master_configs:
            raise Exception(f"No se encontró configuración de maestros en {TABLE_CONFIG}")

        # --- 2. Recolectar todos los DataFrames de Staging ---
        dfs_to_union = []
        staging_table_paths = [] # Guardar las rutas para la limpieza final
        
        print(f"Recolectando datos de {len(master_configs)} tablas de staging...")

        # Obtenemos los execution_id que YA están en la tabla final
        df_processed_ids = spark.table(EVIDENCES_TABLE).select("execution_id").distinct()

        for config in master_configs:
            staging_table = config.staging_evidences_table
            staging_table_path = f"{CATALOG}.{SCHEMA}.{staging_table}"
            staging_table_paths.append(staging_table_path) 
            
            if spark.catalog.tableExists(staging_table_path):
                print(f"  > Leyendo datos de: {staging_table_path}")
                df_staging = spark.table(staging_table_path)
                
                if df_staging.head(1):
                    df_filtered = df_staging.join(
                        broadcast(df_processed_ids),
                        "execution_id",
                        "left_anti"
                    )
                    if df_filtered.head(1):
                        dfs_to_union.append(df_filtered)
                    else:
                        print(f"  > Todas las filas de {staging_table_path} ya procesadas. Saltando.")
                else:
                    print(f"  > AVISO: La tabla {staging_table_path} existe pero está vacía. Saltando.")
            else:
                print(f"  > AVISO: No se encontró la tabla {staging_table_path}. Saltando.")

        # --- 3. Unificar DataFrames ---
        df_staging_completo: DataFrame = dfs_to_union[0]
        for df in dfs_to_union[1:]:
            df_staging_completo = df_staging_completo.unionByName(df, allowMissingColumns=True)
        # Eliminamos duplicados
        df_staging_completo = df_staging_completo.distinct()#.cache()


        # --- 4. Execution_id realmente nuevos ---
        # Obtenemos todos los execution_id que han llegado a staging
        exec_ids_in_staging = df_staging_completo.select("execution_id").distinct()
        # Filtramos 'exec_ids_in_staging' para quedarnos solo con los que
        # NO están en 'df_processed_ids' (LEFT_ANTI JOIN)
        df_unprocessed_ids = exec_ids_in_staging.join(
            broadcast(df_processed_ids),
            "execution_id",
            "left_anti"
        )#.cache()

        unprocessed_count = df_unprocessed_ids.count()
        if unprocessed_count == 0:
            print("No hay ejecuciones nuevas después de filtrar ya procesadas.")
            return 0

        # Filtrar solo datos nuevos
        df_datos_nuevos = df_staging_completo.join(
            broadcast(df_unprocessed_ids),
            "execution_id",
            "inner"
        )
        total_rows = df_datos_nuevos.count()

        # --- 5. Append a tabla final ---
        if total_rows > 0:
            print(f"Añadiendo {total_rows} filas nuevas a {EVIDENCES_TABLE}")
            df_datos_nuevos.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable(EVIDENCES_TABLE)
            print(f"¡Unificación completada!")
        else:
            print("No se encontraron registros nuevos para añadir a la tabla final.")


        # --- 6. Limpieza segura de staging ---
        for table_path in staging_table_paths:
            if spark.catalog.tableExists(table_path):
                try:                  
                    spark.sql(f"DROP TABLE {table_path}")
                    print(f"  > Tabla eliminada correctamente: {table_path}")
                except Exception as e:
                    print(f"  > AVISO: Fallo al limpiar {table_path}: {e}")
    
    except Exception as e:
        print(f"Error fatal durante la unificación de evidencias: {e}")
        if 'df_unprocessed_ids' in locals() and df_unprocessed_ids.is_cached:
            df_unprocessed_ids.unpersist()
        raise e
    
    finally:
        #df_unprocessed_ids.unpersist()
        print("comentado")
    
    return total_rows



# COMMAND ----------

# DBTITLE 3, 3. Punto de Entrada de Ejecución
if __name__ == "__main__":
    rows_processed = 0
    try:
        # Ejecutamos la función principal
        rows_processed = main()
    except Exception as e:
        # Salida explícita en caso de fallo
        dbutils.notebook.exit(f"Fallo en la unificación de evidencias: {e}")
    else:
        # Salida explícita en caso de éxito
        dbutils.notebook.exit(f"Éxito: {rows_processed} evidencias nuevas unificadas y añadidas.")
